In [33]:
import os

subj = "Subj1"
N_STAGES = 9

N_SELECTION_STAGES = 5
NUM_FEATURES = 765
RERUN = False

exp = "exp_select_features"
os.makedirs(f"{subj}/{exp}", exist_ok = True)

In [34]:
%load_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings('ignore')

import SDA
import SDA.analytics
import SDA.clustquality

import umap
import tqdm
import numpy
import pandas
import sklearn.metrics
import sklearn.preprocessing
import sklearn.decomposition
import tqdm.contrib.itertools
import sklearn.feature_selection
import sklearn.cross_decomposition

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [35]:
def explained_variance(features, reduced):
    pls = sklearn.cross_decomposition.PLSRegression(n_components = reduced.shape[1])
    y_pred = pls.fit(reduced, features).predict(reduced)
    return sklearn.metrics.r2_score(features, y_pred, multioutput = "variance_weighted")

### Selection

In [36]:
edges_true = numpy.loadtxt(f"{subj}/reproduction/internal/best_edges.txt").astype(numpy.int32)
df_features = pandas.read_feather(f'{subj}/exp_final_filtered/all_features.feather')
print(df_features.shape)

features = sklearn.preprocessing.StandardScaler().fit_transform(df_features)
print(features.shape)

params = {
    'n_clusters_min': 8, 'n_clusters_max': 8,
    'k_neighbours_min': 20, 'k_neighbours_max': 35,
    'len_st_thr': [ 20 ],

    'n_cl_max_thr': [ 8 ],
    'k_neighb_max_thr': [ 35 ],
    'n_edge_clusters_min': N_SELECTION_STAGES - 1, 'n_edge_clusters_max': N_SELECTION_STAGES - 1
}

(1046, 3799)
(1046, 3799)


In [37]:
if RERUN:
    scores = [ ]
    for i in tqdm.trange(features.shape[1]):
        try:
            result, _ = SDA.SDA(**params, n_jobs = 15, scale = False, verbose = False).apply(features[:, i].reshape(-1, 1))
            result = SDA.analytics.best_result(result, key = 'Avg-Silh', n_stages = N_SELECTION_STAGES)
            if len(result['St_edges']) != N_SELECTION_STAGES + 1:
                raise RuntimeError()
            score = result['Avg-Silh']
        except:
            score = -1
        scores.append({ 'index': i, 'name': df_features.columns[i], 'score': score })
    scores = pandas.DataFrame(scores)
else:
    scores = pandas.read_csv(f"{subj}/{exp}/scores.csv")
display(scores)

,index,name,score
0,0,channel-0 entropy dim-1,0.160363
1,1,channel-0 entropy dim-2,0.132280
2,2,channel-0 numberofpoints dim-1,0.182975
3,3,channel-0 numberofpoints dim-2,0.183828
4,4,channel-0 amplitude-bottleneck dim-1,0.472430
...,...,...,...
3794,3794,overall bd2 dim-3 mean,0.075003
3795,3795,overall bd2 dim-3 std,0.023684
3796,3796,overall bd2 dim-3 sum,0.155738
3797,3797,overall bd2 dim-3 norm-1,0.155738


In [38]:
scores.to_csv(f"{subj}/{exp}/scores.csv", index = False)
display(scores.sort_values(by = 'score', ascending = False))

,index,name,score
2437,2437,channel-26 amplitude-silhouette-1-2 dim-2,0.578132
2053,2053,channel-22 amplitude-landscape-1-2 dim-2,0.558201
2441,2441,channel-26 amplitude-silhouette-2-1 dim-2,0.533642
2347,2347,channel-25 amplitude-silhouette-1-2 norm-2,0.506922
2324,2324,channel-25 amplitude-landscape-1-1 dim-1,0.490045
...,...,...,...
2346,2346,channel-25 amplitude-silhouette-1-2 norm-1,-1.000000
1959,1959,channel-21 amplitude-landscape-1-1 norm-2,-1.000000
89,89,channel-0 bd2 dim-2 sum,-1.000000
1144,1144,channel-12 amplitude-silhouette-1-1 dim-1,-1.000000


In [39]:
best_feature_idx = scores.sort_values(by = 'score', ascending = False)[:NUM_FEATURES]["index"].to_numpy()
best_features = features[:, best_feature_idx]
print(best_features.shape)

(1046, 765)


### UMAP

In [40]:
tda_umap = umap.UMAP(n_components = 15, random_state = 42)
umap_features = tda_umap.fit_transform(best_features)
print(umap_features.shape)

(1046, 15)


In [41]:
umap_result, _ = SDA.SDA(n_jobs = 15, scale = False, verbose = True).apply(umap_features)
display(SDA.analytics.best_results(umap_result, key = 'Avg-Silh'))

Applying to 1046 samples with 15 features each
Running stage 1


  0%|          | 0/589 [00:00<?, ?it/s]

Running stage 2


  0%|          | 0/672 [00:00<?, ?it/s]

,St_len_min,K_nb_max,N_cl_max,N_stages,Cl_cen,St_edges,Ward_dist,Cen_dist,Silh,Cal-Har,Dav-Bold,Avg-Silh,Avg-Cal-Har,Avg-Dav-Bold
0,0,35,10,3,Mode,"[0, 290, 682, 1046]",3161.919283,3.474061,0.215219,860.349035,3.395348,0.333131,850.619175,2.663925
1,0,35,15,4,Mode,"[0, 290, 682, 787, 1046]",2573.245531,4.112788,0.200933,755.632592,5.558532,0.404413,700.134729,0.755719
2,0,50,10,5,Mode,"[0, 290, 554, 682, 787, 1046]",1791.529439,4.181176,0.225177,801.903369,1.843722,0.515889,779.809101,0.661098
3,0,35,20,6,Median,"[0, 280, 556, 682, 795, 976, 1046]",1546.590371,3.872199,0.207794,738.661037,2.906555,0.468554,683.577151,0.827008
4,0,35,20,7,Median,"[0, 280, 556, 682, 787, 856, 976, 1046]",1258.147809,3.647017,0.195297,734.306928,3.642418,0.500515,592.476122,0.739815
5,60,40,20,8,Median,"[0, 190, 290, 556, 682, 787, 856, 976, 1046]",688.150690,3.224527,0.113321,703.074177,3.236455,0.468854,347.111136,0.814997
6,0,35,15,9,Mode,"[0, 55, 244, 290, 556, 682, 787, 856, 976, 1046]",495.462799,2.917004,0.066165,643.867856,3.057780,0.433679,262.338960,0.908948
7,60,45,15,10,Median,"[0, 199, 290, 509, 556, 664, 682, 787, 856, 97...",317.153436,2.645310,0.122616,580.820142,3.814688,0.414563,166.750233,1.074652
8,20,45,15,11,Median,"[0, 55, 244, 290, 509, 556, 665, 682, 787, 856...",221.374642,2.459239,0.066885,544.435407,3.727175,0.387424,122.004968,1.138406
9,20,45,15,12,Median,"[0, 55, 190, 244, 290, 509, 556, 665, 682, 787...",189.069766,2.259472,0.058645,510.487643,3.566863,0.362128,105.185977,1.195828


In [42]:
umap_best_result = SDA.analytics.best_result(umap_result, key = 'Avg-Silh', n_stages = N_STAGES)
umap_edges = umap_best_result['St_edges']

print('Features:', features.shape)
print('Best features:', best_features.shape)
print('UMAP features:', umap_features.shape)

print('Explained variance 15-765:', explained_variance(best_features, umap_features))
print('Explained variance 15-3799:', explained_variance(features, umap_features))

print('Outer:', SDA.clustquality.cluster_metrics_ground(edges_true, umap_edges))

print('Inner (15):', SDA.clustquality.calc_stage_metr_noground(umap_features, umap_edges).mean().to_dict())
print('Inner (765):', SDA.clustquality.calc_stage_metr_noground(best_features, umap_edges).mean().to_dict())
print('Inner (3799):', SDA.clustquality.calc_stage_metr_noground(features, umap_edges).mean().to_dict())

Features: (1046, 3799)
Best features: (1046, 765)
UMAP features: (1046, 15)
Explained variance 15-765: 0.5302920828684512
Explained variance 15-3799: 0.3470002515371061
Outer: {'AMI': 0.8893655855295552, 'ARI': 0.7966591496360627, 'FMI': 0.8264222076670673}
Inner (15): {'Silh': 0.43367886543273926, 'Cal-Har': 262.33896021593057, 'Dav-Bold': 0.9089476078874245}
Inner (765): {'Silh': 0.15946849726382195, 'Cal-Har': 48.93918855758832, 'Dav-Bold': 2.281237016684232}
Inner (3799): {'Silh': 0.10235849560290305, 'Cal-Har': 28.88107534905314, 'Dav-Bold': 3.1293096465696544}


### PCA

In [43]:
pca = sklearn.decomposition.PCA(n_components = 15, svd_solver = "full", random_state = 42)
pca_features = pca.fit_transform(best_features)

print('Explained variance', round(pca.explained_variance_ratio_.sum(), 2))
print([ round(x, 3) for x in pca.explained_variance_ratio_ ])
print(pca_features.shape)
print(explained_variance(best_features, pca_features))

Explained variance 0.75
[0.294, 0.111, 0.083, 0.051, 0.034, 0.03, 0.023, 0.023, 0.018, 0.018, 0.015, 0.013, 0.013, 0.013, 0.012]
(1046, 15)
0.7509675089020927


In [44]:
pca_result, _ = SDA.SDA(n_jobs = 15, scale = False, verbose = True).apply(pca_features)
display(SDA.analytics.best_results(pca_result, key = 'Avg-Silh'))

Applying to 1046 samples with 15 features each
Running stage 1


  0%|          | 0/589 [00:00<?, ?it/s]

Running stage 2


  0%|          | 0/672 [00:00<?, ?it/s]

,St_len_min,K_nb_max,N_cl_max,N_stages,Cl_cen,St_edges,Ward_dist,Cen_dist,Silh,Cal-Har,Dav-Bold,Avg-Silh,Avg-Cal-Har,Avg-Dav-Bold
0,60,40,15,3,Mode,"[0, 210, 556, 1046]",42015.483009,15.762431,0.061632,136.386109,3.418155,0.155654,109.947265,2.930341
1,40,40,20,4,Mode,"[0, 210, 682, 795, 1046]",58313.368541,22.714365,0.050441,120.938596,4.623428,0.209465,135.506905,1.555254
2,40,50,15,5,Mode,"[0, 210, 556, 682, 795, 1046]",57059.170329,25.675386,0.068703,137.352410,2.827397,0.288836,155.834511,1.383560
3,60,40,20,6,Mode,"[0, 210, 556, 682, 856, 976, 1046]",52141.731206,25.177458,0.054235,124.617161,3.210948,0.295695,154.401042,1.331537
4,20,45,20,7,Mode,"[0, 55, 210, 556, 682, 795, 976, 1046]",40594.731614,22.710237,0.042044,106.395345,3.105567,0.254761,117.979753,1.629336
5,60,45,20,8,Median,"[0, 210, 310, 556, 682, 789, 856, 976, 1046]",27577.629798,20.732497,0.022504,102.786262,3.583614,0.239136,85.531844,1.655676
6,40,45,20,9,Median,"[0, 55, 210, 310, 556, 682, 789, 856, 976, 1046]",24828.727764,19.856665,0.019410,94.780000,3.517684,0.224941,76.753998,1.754172
7,60,40,20,10,Median,"[0, 194, 210, 250, 313, 556, 682, 795, 856, 97...",19524.170155,18.335139,-0.016890,82.094757,3.605503,0.216702,62.023352,1.901630
8,0,40,20,11,Mode,"[0, 55, 141, 210, 313, 556, 659, 795, 856, 976...",18315.209353,18.748918,-0.000040,73.127844,3.779201,0.213725,55.651415,2.141425
9,0,45,20,12,Mode,"[0, 55, 141, 210, 313, 489, 556, 682, 795, 856...",15092.483590,17.662368,-0.006272,72.756745,3.771718,0.191680,47.703967,2.198586


In [45]:
pca_best_result = SDA.analytics.best_result(pca_result, key = 'Avg-Silh', n_stages = N_STAGES)
pca_edges = pca_best_result['St_edges']

print('Features:', features.shape)
print('Best features:', best_features.shape)
print('PCA features:', pca_features.shape)

print('Explained variance 15-765:', explained_variance(best_features, pca_features))
print('Explained variance 15-3799:', explained_variance(features, pca_features))

print('Outer:', SDA.clustquality.cluster_metrics_ground(edges_true, pca_edges))

print('Inner (15):', SDA.clustquality.calc_stage_metr_noground(pca_features, pca_edges).mean().to_dict())
print('Inner (765):', SDA.clustquality.calc_stage_metr_noground(best_features, pca_edges).mean().to_dict())
print('Inner (3799):', SDA.clustquality.calc_stage_metr_noground(features, pca_edges).mean().to_dict())

Features: (1046, 3799)
Best features: (1046, 765)
PCA features: (1046, 15)
Explained variance 15-765: 0.7509675089020927
Explained variance 15-3799: 0.4780859208841952
Outer: {'AMI': 0.8598368942534087, 'ARI': 0.7264662152028356, 'FMI': 0.7650891816151753}
Inner (15): {'Silh': 0.22494147567011816, 'Cal-Har': 76.75399835955376, 'Dav-Bold': 1.7541718053147854}
Inner (765): {'Silh': 0.15828494195550596, 'Cal-Har': 50.61578221066756, 'Dav-Bold': 2.261939794151778}
Inner (3799): {'Silh': 0.09501956035888828, 'Cal-Har': 28.966490927729804, 'Dav-Bold': 3.1772510779755825}


### Traditional features

In [46]:
df_ft_psd_loc_db = pandas.read_feather(f'{subj}/src/df_ft_psd_loc_db.feather')
df_ft_psd_ind_loc_log = pandas.read_feather(f'{subj}/src/df_ft_psd_ind_loc_log.feather')
df_ft_coh_ind_loc = pandas.read_feather(f'{subj}/src/df_ft_coh_ind_loc.feather')
df_ft_plv_ind_loc = pandas.read_feather(f'{subj}/src/df_ft_plv_ind_loc.feather')

features_neuro = pandas.concat([ df_ft_psd_loc_db, df_ft_psd_ind_loc_log, df_ft_coh_ind_loc, df_ft_plv_ind_loc ], axis = 1)
print(features_neuro.shape)

features_neuro = sklearn.preprocessing.StandardScaler().fit_transform(features_neuro)
print(features_neuro.shape)

(1046, 765)
(1046, 765)


#### UMAP

In [47]:
neuro_umap = umap.UMAP(n_components = 15, random_state = 42)
features_neuro_umap = neuro_umap.fit_transform(features_neuro)
print(features_neuro_umap.shape)

(1046, 15)


In [48]:
neuro_umap_result, _ = SDA.SDA(n_jobs = 15, scale = False, verbose = True).apply(features_neuro_umap)
display(SDA.analytics.best_results(neuro_umap_result, key = 'Avg-Silh'))

Applying to 1046 samples with 15 features each
Running stage 1


  0%|          | 0/589 [00:00<?, ?it/s]

Running stage 2


  0%|          | 0/672 [00:00<?, ?it/s]

,St_len_min,K_nb_max,N_cl_max,N_stages,Cl_cen,St_edges,Ward_dist,Cen_dist,Silh,Cal-Har,Dav-Bold,Avg-Silh,Avg-Cal-Har,Avg-Dav-Bold
0,0,35,15,3,Median,"[0, 232, 681, 1046]",2337.918949,3.473309,0.225711,567.432745,1.859608,0.332809,606.796648,1.545486
1,0,35,10,4,Mode,"[0, 263, 681, 777, 1046]",2692.104897,5.128663,0.249266,754.597017,2.894817,0.587176,804.180915,0.554250
2,0,35,10,5,Mode,"[0, 263, 555, 681, 857, 1046]",1582.330946,3.882507,0.239389,589.867979,1.457409,0.453061,533.176090,0.871468
3,40,45,15,6,Median,"[0, 263, 555, 681, 777, 976, 1046]",1529.732542,4.037761,0.218712,684.150707,1.750357,0.494924,695.206290,0.910106
4,0,35,10,7,Median,"[0, 103, 263, 555, 681, 777, 976, 1046]",1074.482228,3.629237,0.183207,712.889985,1.641383,0.478156,547.630230,0.904382
5,0,35,10,8,Median,"[0, 103, 263, 555, 681, 777, 857, 976, 1046]",871.482210,3.530660,0.195523,739.037591,2.466327,0.519981,502.692844,0.727290
6,0,35,10,9,Median,"[0, 103, 263, 492, 555, 681, 777, 857, 976, 1046]",715.690314,3.341220,0.228852,756.272604,2.270272,0.510964,452.897032,0.736405
7,0,35,15,10,Median,"[0, 103, 194, 263, 492, 555, 681, 777, 857, 97...",525.656640,2.982591,0.217626,735.812963,2.173425,0.487485,348.323992,0.794628
8,0,35,15,11,Median,"[0, 39, 103, 194, 263, 492, 555, 681, 777, 857...",468.736672,2.785580,0.223040,693.776986,2.035597,0.495106,325.403070,0.780897
9,0,35,10,12,Median,"[0, 39, 103, 194, 232, 263, 492, 555, 681, 777...",399.733147,2.548545,0.212193,635.193597,2.226929,0.460038,276.303980,0.959830


In [49]:
neuro_umap_best_result = SDA.analytics.best_result(neuro_umap_result, key = 'Avg-Silh', n_stages = N_STAGES)
neuro_umap_edges = neuro_umap_best_result['St_edges']

print('Neuro features:', features_neuro.shape)
print('UMAP neuro features:', features_neuro_umap.shape)

print('Explained variance 15-765:', explained_variance(features_neuro, features_neuro_umap))

print('Outer:', SDA.clustquality.cluster_metrics_ground(edges_true, neuro_umap_edges))

print('Inner (15):', SDA.clustquality.calc_stage_metr_noground(features_neuro_umap, neuro_umap_edges).mean().to_dict())
print('Inner (765):', SDA.clustquality.calc_stage_metr_noground(features_neuro, neuro_umap_edges).mean().to_dict())

Neuro features: (1046, 765)
UMAP neuro features: (1046, 15)
Explained variance 15-765: 0.50961167309074
Outer: {'AMI': 0.8989658031918278, 'ARI': 0.8201120691706356, 'FMI': 0.8459071662662822}
Inner (15): {'Silh': 0.5109639167785645, 'Cal-Har': 452.89703155772366, 'Dav-Bold': 0.7364054709635279}
Inner (765): {'Silh': 0.13754202043469593, 'Cal-Har': 42.38118519698051, 'Dav-Bold': 2.238258625900672}


#### PCA

In [50]:
neuro_pca = sklearn.decomposition.PCA(n_components = 15, svd_solver = "full", random_state = 42)
features_neuro_pca = neuro_pca.fit_transform(features_neuro)

print('Explained variance', round(neuro_pca.explained_variance_ratio_.sum(), 2))
print([ round(x, 3) for x in neuro_pca.explained_variance_ratio_ ])
print(features_neuro_pca.shape)
print(explained_variance(features_neuro, features_neuro_pca))

Explained variance 0.71
[0.211, 0.156, 0.069, 0.063, 0.046, 0.032, 0.026, 0.02, 0.018, 0.015, 0.013, 0.012, 0.01, 0.009, 0.009]
(1046, 15)
0.7106773827080092


In [51]:
neuro_pca_result, _ = SDA.SDA(n_jobs = 15, scale = False, verbose = True).apply(features_neuro_pca)
display(SDA.analytics.best_results(neuro_pca_result, key = 'Avg-Silh'))

Applying to 1046 samples with 15 features each
Running stage 1


  0%|          | 0/589 [00:00<?, ?it/s]

Running stage 2


  0%|          | 0/672 [00:00<?, ?it/s]

,St_len_min,K_nb_max,N_cl_max,N_stages,Cl_cen,St_edges,Ward_dist,Cen_dist,Silh,Cal-Har,Dav-Bold,Avg-Silh,Avg-Cal-Har,Avg-Dav-Bold
0,0,35,10,3,Mode,"[0, 282, 560, 1046]",42308.816307,16.400094,0.072681,114.988709,2.722181,0.125930,108.777275,2.438303
1,20,40,15,4,Median,"[0, 168, 560, 857, 1046]",33307.136107,15.081889,0.070873,89.780730,3.560225,0.127895,86.288835,3.119853
2,40,50,10,5,Median,"[0, 178, 560, 682, 857, 1046]",45198.297789,21.835308,0.084183,105.595294,2.586946,0.211379,121.129124,1.797900
3,0,35,20,6,Mode,"[0, 39, 282, 560, 682, 857, 1046]",34637.660711,20.807593,0.058036,91.372564,2.433889,0.189075,94.179967,1.796114
4,0,40,20,7,Mode,"[0, 39, 282, 560, 682, 857, 976, 1046]",33803.157325,21.727724,0.027373,87.867345,3.085920,0.197685,90.699464,1.694186
5,60,45,20,8,Median,"[0, 104, 277, 557, 682, 784, 857, 976, 1046]",27148.763733,20.782267,0.051638,88.107028,3.174767,0.202978,77.226016,1.764923
6,0,45,20,9,Mode,"[0, 39, 282, 492, 560, 682, 784, 857, 976, 1046]",24209.602081,21.382082,0.040342,81.951371,2.922261,0.199396,69.908675,1.637018
7,20,40,15,10,Mode,"[0, 92, 154, 282, 492, 560, 682, 784, 857, 976...",19921.966427,19.978023,0.055157,80.814883,2.941300,0.197371,59.065692,1.797313
8,40,40,15,11,Mode,"[0, 92, 154, 282, 492, 560, 609, 682, 784, 857...",17433.673781,20.010478,0.047344,76.533681,2.935801,0.205210,56.018632,1.818006
9,40,40,20,12,Mode,"[0, 95, 154, 282, 492, 560, 609, 682, 784, 857...",15496.871841,19.770975,0.048565,72.453995,2.928984,0.204760,51.143214,1.887942


In [52]:
neuro_pca_best_result = SDA.analytics.best_result(neuro_pca_result, key = 'Avg-Silh', n_stages = N_STAGES)
neuro_pca_edges = neuro_pca_best_result['St_edges']

print('Neuro features:', features_neuro.shape)
print('PCA neuro features:', features_neuro_pca.shape)

print('Explained variance 15-765:', explained_variance(features_neuro, features_neuro_pca))

print('Outer:', SDA.clustquality.cluster_metrics_ground(edges_true, neuro_pca_edges))

print('Inner (15):', SDA.clustquality.calc_stage_metr_noground(features_neuro_pca, neuro_pca_edges).mean().to_dict())
print('Inner (765):', SDA.clustquality.calc_stage_metr_noground(features_neuro, neuro_pca_edges).mean().to_dict())

Neuro features: (1046, 765)
PCA neuro features: (1046, 15)
Explained variance 15-765: 0.7106773827080092
Outer: {'AMI': 1.0, 'ARI': 1.0, 'FMI': 1.0}
Inner (15): {'Silh': 0.1993961405684552, 'Cal-Har': 69.90867530727269, 'Dav-Bold': 1.637017876621227}
Inner (765): {'Silh': 0.13823275074674224, 'Cal-Har': 44.432973449387084, 'Dav-Bold': 2.115450457714017}
